In [1]:
import logging
import sys
import numpy as np 
import pandas as pd
from pandas import to_datetime
import os
import glob
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logger = logging.getLogger('LOGGER_NAME')
logger.info('Start CambiumAnomalyClassificator')

INFO:LOGGER_NAME:Start CambiumAnomalyClassificator


# Load data from Cambium CPEs reports in a single file

In [2]:

def get_concat_csv(flist, **kwargs):
    return pd.concat([pd.read_csv(f, **kwargs) for f in flist], axis=0).reset_index()

path = './dataset cambium/'

fmask = os.path.join(path, '*.csv')
print(fmask)
list_of_files=glob.glob(fmask)
print(list_of_files)
Data_Cambium = get_concat_csv(glob.glob(fmask),sep = ',', encoding='utf-8')

print(Data_Cambium.shape)
print(Data_Cambium.info())
print(Data_Cambium)

logger.info("Saving a single Dataset")
Data_Cambium.to_csv('./Single_Dataset_Cambium.csv', encoding='utf-8')


./dataset cambium/*.csv
['./dataset cambium\\cnMaestro-cnmaestro_on_premises-performance-24Hours-ePMP-system-daily-20201225T000000-21d21a70-4644-11eb-abb7-d137dbf3e514.csv', './dataset cambium\\cnMaestro-cnmaestro_on_premises-performance-24Hours-ePMP-system-daily-20201226T000000-4c239780-470d-11eb-abb7-d137dbf3e514.csv', './dataset cambium\\cnMaestro-cnmaestro_on_premises-performance-24Hours-ePMP-system-daily-20201227T000000-76cfb7b0-47d6-11eb-abb7-d137dbf3e514.csv', './dataset cambium\\cnMaestro-cnmaestro_on_premises-performance-24Hours-ePMP-system-daily-20201228T000000-a11e2780-489f-11eb-abb7-d137dbf3e514.csv', './dataset cambium\\cnMaestro-cnmaestro_on_premises-performance-24Hours-ePMP-system-daily-20201229T000000-cb83efe0-4968-11eb-abb7-d137dbf3e514.csv', './dataset cambium\\cnMaestro-cnmaestro_on_premises-performance-24Hours-ePMP-system-daily-20201230T000000-f5f57810-4a31-11eb-abb7-d137dbf3e514.csv', './dataset cambium\\cnMaestro-cnmaestro_on_premises-performance-24Hours-ePMP-syst

# Loading dataset 

Features are: Polling Timestamp,MAC,Device Name, Device Type, Device Mode, Downlink MCS, Uplink MCS, Downlink ThroughPut (Kbps), Uplink ThroughPut (Kbps), Downlink Retransmission Pkts (%),
Uplink Retransmission Pkts (%), CPUs, Session Drops, Downlink SNR (dB), Uplink SNR (dB),
Downlink Usage (Packets), Uplink Usage (Packets), Downlink RSSI (dBm), Uplink RSSI (dBm),
Offline Count, Status, Status Duration.

In [3]:
data = './Single_Dataset_Cambium.csv'
colX = ['Polling Timestamp', 'MAC', 'Device Name', 'Device Type', 'Device Mode',
        'Downlink MCS', 'Uplink MCS', 'Downlink ThroughPut (Kbps)', 'Uplink ThroughPut (Kbps)',
        'Downlink Retransmission Pkts (%)', 'Uplink Retransmission Pkts (%)', 'CPUs',
        'Session Drops', 'Downlink SNR (dB)', 'Uplink SNR (dB)', 'Downlink Usage (Packets)',
        'Uplink Usage (Packets)', 'Downlink RSSI (dBm)', 'Uplink RSSI (dBm)', 'Offline Count',
        'Status', 'Status Duration']

X_data_cambium = pd.read_csv(data, sep = ',', usecols= colX, encoding='utf-8') 
print(X_data_cambium.info())
print(X_data_cambium.shape)
print(X_data_cambium)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46280 entries, 0 to 46279
Data columns (total 22 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Polling Timestamp                 46280 non-null  object 
 1   MAC                               46280 non-null  object 
 2   Device Name                       46280 non-null  object 
 3   Device Type                       46280 non-null  object 
 4   Device Mode                       46280 non-null  object 
 5   Downlink MCS                      46280 non-null  object 
 6   Uplink MCS                        46280 non-null  object 
 7   Downlink ThroughPut (Kbps)        46280 non-null  float64
 8   Uplink ThroughPut (Kbps)          46280 non-null  float64
 9   Downlink Retransmission Pkts (%)  46280 non-null  float64
 10  Uplink Retransmission Pkts (%)    46280 non-null  float64
 11  CPUs                              46280 non-null  float64
 12  Sess

# Selecting records with "Device Mode" = "SM".

In [4]:
print ('Dataset before dropping records:\n', X_data_cambium.shape)
print(X_data_cambium ['Device Mode'].value_counts())
X_data_cambium = X_data_cambium.drop(X_data_cambium.loc[(X_data_cambium['Device Mode'] != 'SM')].index)
print ('Dataset after dropping records:\n', X_data_cambium.shape)
print(X_data_cambium ['Device Mode'].value_counts())

Dataset before dropping records:
 (46280, 22)
SM    43780
AP     2500
Name: Device Mode, dtype: int64
Dataset after dropping records:
 (43780, 22)
SM    43780
Name: Device Mode, dtype: int64


In [5]:
# Status Duration: calculating amount of minutes

print(X_data_cambium.info())

MStatusDuration = []
for row in X_data_cambium['Status Duration']:
    #print(row)
    temp = (str(row)).split()
    #print(temp)
    d = ((temp[0].split('d'))[0])
    h = (temp[1].split('h'))[0]
    m = (temp[2].split('m'))[0]
    #print(int(d)*24*60+int(h)*60+int(m))
    MStatusDuration.append(int(d)*24*60+int(h)*60+int(m))
X_data_cambium['MStatus Duration'] = MStatusDuration
# elimina caratteristica origine
X_data_cambium.drop(columns=['Status Duration'],inplace=True)
print (X_data_cambium)

print(X_data_cambium.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 43780 entries, 0 to 46279
Data columns (total 22 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Polling Timestamp                 43780 non-null  object 
 1   MAC                               43780 non-null  object 
 2   Device Name                       43780 non-null  object 
 3   Device Type                       43780 non-null  object 
 4   Device Mode                       43780 non-null  object 
 5   Downlink MCS                      43780 non-null  object 
 6   Uplink MCS                        43780 non-null  object 
 7   Downlink ThroughPut (Kbps)        43780 non-null  float64
 8   Uplink ThroughPut (Kbps)          43780 non-null  float64
 9   Downlink Retransmission Pkts (%)  43780 non-null  float64
 10  Uplink Retransmission Pkts (%)    43780 non-null  float64
 11  CPUs                              43780 non-null  float64
 12  Sess

# Selecting records with "Status" = "Online" or  ("Status" = "Offline" and MStatus Duration < 10gg)

In [6]:
print ('Dataset before dropping records:\n', X_data_cambium.shape)
print(X_data_cambium ['Status'].value_counts())
X_data_cambium = X_data_cambium.drop(X_data_cambium.loc[(X_data_cambium['Status'] == 'Offline') & (X_data_cambium['MStatus Duration'] > 14400)].index)
print ('Dataset after dropping records:\n', X_data_cambium.shape)
print(X_data_cambium ['Status'].value_counts())


Dataset before dropping records:
 (43780, 22)
Online     30870
Offline    12910
Name: Status, dtype: int64
Dataset after dropping records:
 (32068, 22)
Online     30870
Offline     1198
Name: Status, dtype: int64


# Saving Dataset 

In [8]:
logger.info("Saving Dataset")
X_data_cambium.to_csv('./X_Data_Cambium.csv', encoding='utf-8')

INFO:LOGGER_NAME:Saving Dataset


In [9]:
logger.info("Printing NaN values:")
print(X_data_cambium.isnull().sum().sum())
if (X_data_cambium.isnull().sum().sum()>0):    
    print(X_data_cambium.isnull().sum())

INFO:LOGGER_NAME:Printing NaN values:
0


# Load file with support requests

In [11]:
data = './ticket/ticket_split.csv'
colX = ['Cliente','DataTime']
X_data_ticket = pd.read_csv(data, sep = ';', usecols= colX, encoding='utf-8') 
print(X_data_ticket.info())
print(X_data_ticket.shape)
print(X_data_ticket)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1714 entries, 0 to 1713
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Cliente   1714 non-null   object
 1   DataTime  1714 non-null   object
dtypes: object(2)
memory usage: 26.9+ KB
None
(1714, 2)
                                       Cliente                   DataTime
0                            ALTOBELLI ALFREDO  2019-02-19 11:30:26+01:00
1                              DOMUS LILIA SRL  2019-03-07 14:52:41+01:00
2          PICCOLO RAFFAELA VIA SANTA FILOMENA  2019-04-18 11:10:40+02:00
3      FRANCHINI CENTRO AUTOD. DI FRANCHINI E.  2019-06-10 12:27:34+02:00
4                                 ZARA NEMESIO  2019-06-17 09:16:17+02:00
...                                        ...                        ...
1709                         MERCURIO BEATRICE  2021-03-22 11:16:54+01:00
1710                             CESARO ANTIMO  2021-03-22 15:02:33+01:00
1711                

In [12]:
print('Printing Polling Timestamp Min,Max :',X_data_cambium['Polling Timestamp'].values.min(),X_data_cambium['Polling Timestamp'].values.max())
print('Printing DataTime Min,Max :',X_data_ticket['DataTime'].values.min(),X_data_ticket['DataTime'].values.max())

Printing Polling Timestamp Min,Max : 2020-12-24 01:00:00 2021-03-22 01:00:00
Printing DataTime Min,Max : 2019-02-19 11:30:26+01:00 2021-03-23 09:12:09+01:00


In [13]:
print(X_data_ticket ['Cliente'].value_counts())
X_data_ticket ['Cliente'] = X_data_ticket ['Cliente'].str.strip()
X_data_ticket ['Cliente'] = X_data_ticket ['Cliente'].str.upper()
print(X_data_ticket ['Cliente'].value_counts())

logger.info("Get only Data from X_data_ticket")
X_data_ticket['DataTime'] = to_datetime(X_data_ticket['DataTime'],dayfirst = True, utc=True)
X_data_ticket['DataTime'] = X_data_ticket['DataTime'].dt.date
X_data_ticket.sort_values(by=['DataTime'], ascending = 1, inplace = True)
print(X_data_ticket)

print(X_data_cambium ['Device Name'].value_counts())
X_data_cambium ['Device Name'] = X_data_cambium ['Device Name'].str.strip()
X_data_cambium ['Device Name'] = X_data_cambium ['Device Name'].str.upper()
print(X_data_cambium ['Device Name'].value_counts())

logger.info("Get only Data from X_data_cambium")
X_data_cambium['Polling Timestamp'] = to_datetime(X_data_cambium['Polling Timestamp'],dayfirst = True, utc=True)
X_data_cambium['Polling Timestamp'] = X_data_cambium['Polling Timestamp'].dt.date
X_data_cambium.sort_values(by=['Polling Timestamp'], ascending = 1, inplace = True)
print(X_data_cambium)

 ESPOSITO GENNARO                              14
 D'ANGIOLELLA PATRIZIO NOTAIO STUDIO AVERSA    14
 CROCIANI MARIA                                12
 GIOVISI SRL                                   11
 GRAZIANO & CO. SRL                            10
                                               ..
 D'ERRICO LUCIANO                               1
 MIGLIOZZI MARCO                                1
 PALMIERO CARLO MARIA                           1
 BARONE MADDALENA                               1
 IANNONE AMALIA                                 1
Name: Cliente, Length: 701, dtype: int64
ESPOSITO GENNARO                              14
D'ANGIOLELLA PATRIZIO NOTAIO STUDIO AVERSA    14
CROCIANI MARIA                                12
GIOVISI SRL                                   11
GRAZIANO & CO. SRL                            10
                                              ..
FERRO SUD SRL SECONDA UTENZA                   1
BARBATO GIOVANNI (SANT'AGATA)                  1
KABIRU RAIMI ADEW

# Dropping records where X_data_ticket DataTime < minimum of Polling Timestamp

In [14]:
X_data_ticket = X_data_ticket[X_data_ticket['DataTime'] >= pd.to_datetime(X_data_cambium['Polling Timestamp'].values.min(),dayfirst = True, utc=True)]
print(X_data_ticket.shape)
logger.info("Saving Dataset... Ticket")
X_data_ticket.to_csv('./X_data_ticket.csv', encoding='utf-8')

(517, 2)
INFO:LOGGER_NAME:Saving Dataset... Ticket


# Merging CPEs status and issues datasets

In [15]:
print("Dataset X_data_cambium:",X_data_cambium.shape)
print("Dataset X_data_ticket:",X_data_ticket.shape)
XY_data = pd.merge(X_data_cambium,X_data_ticket,left_on=['Device Name','Polling Timestamp'],right_on=['Cliente','DataTime'],how='left')
print ("Dataset XY_data:", XY_data.shape)

Dataset X_data_cambium: (32068, 22)
Dataset X_data_ticket: (517, 2)
Dataset XY_data: (32068, 24)


In [16]:
print ("Dataset XY_data:", XY_data.info())
print ("Dataset XY_data:", XY_data)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32068 entries, 0 to 32067
Data columns (total 24 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Polling Timestamp                 32068 non-null  object 
 1   MAC                               32068 non-null  object 
 2   Device Name                       32068 non-null  object 
 3   Device Type                       32068 non-null  object 
 4   Device Mode                       32068 non-null  object 
 5   Downlink MCS                      32068 non-null  object 
 6   Uplink MCS                        32068 non-null  object 
 7   Downlink ThroughPut (Kbps)        32068 non-null  float64
 8   Uplink ThroughPut (Kbps)          32068 non-null  float64
 9   Downlink Retransmission Pkts (%)  32068 non-null  float64
 10  Uplink Retransmission Pkts (%)    32068 non-null  float64
 11  CPUs                              32068 non-null  float64
 12  Sess

In [17]:
XY_data.to_csv('./XY_data.csv', encoding='utf-8')

# Carico file con Ticket chiusi

In [22]:
data = './ticket/ticket chiusi split.csv'
colX = ['Cliente Ticket Chiuso','DataTime Ticket Chiuso']
X_data_ticket_closed = pd.read_csv(data, sep = ';', usecols= colX, encoding='utf-8') 
print(X_data_ticket_closed.info())
print(X_data_ticket_closed.shape)
print(X_data_ticket_closed)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 403 entries, 0 to 402
Data columns (total 2 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Cliente Ticket Chiuso   403 non-null    object
 1   DataTime Ticket Chiuso  403 non-null    object
dtypes: object(2)
memory usage: 6.4+ KB
None
(403, 2)
    Cliente Ticket Chiuso     DataTime Ticket Chiuso
0           ADZOVIC FEHIM  2021-03-24 11:52:37+01:00
1           ADZOVIC FEHIM  2021-01-29 12:53:54+01:00
2           ADZOVIC FEHIM  2021-03-23 12:33:48+01:00
3            AGOSTI MARIO  2021-01-20 09:43:35+01:00
4             ALETOR JOAN  2021-01-12 09:42:33+01:00
..                    ...                        ...
398   VILLA DEI CEDRI SRL  2021-02-11 16:43:24+01:00
399        XHELAJ BLERINA  2021-01-18 15:08:24+01:00
400      ZAGARIA RAFFAELE  2021-03-17 09:45:01+01:00
401        ZAMPELLA LUIGI  2021-03-03 09:35:12+01:00
402          ZARA NEMESIO  2021-03-01 11:59:44+01:0

In [23]:
print(X_data_ticket_closed ['Cliente Ticket Chiuso'].value_counts())
X_data_ticket_closed ['Cliente Ticket Chiuso'] = X_data_ticket_closed ['Cliente Ticket Chiuso'].str.strip()
X_data_ticket_closed ['Cliente Ticket Chiuso'] = X_data_ticket_closed ['Cliente Ticket Chiuso'].str.upper()
print(X_data_ticket_closed ['Cliente Ticket Chiuso'].value_counts())

logger.info("Get only Data from X_data_ticket_closed")
X_data_ticket_closed['DataTime Ticket Chiuso'] = to_datetime(X_data_ticket_closed['DataTime Ticket Chiuso'],dayfirst = True, utc=True)
X_data_ticket_closed['DataTime Ticket Chiuso'] = X_data_ticket_closed['DataTime Ticket Chiuso'].dt.date
X_data_ticket_closed.sort_values(by=['DataTime Ticket Chiuso'], ascending = 1, inplace = True)
print(X_data_ticket_closed.info())

RAN SRL                                       5
ESPOSITO GENNARO                              5
TROTTA DOMENICO                               5
D'ANGIOLELLA PATRIZIO NOTAIO STUDIO AVERSA    4
FERRANTE ANTONIETTA                           4
                                             ..
GALLO GIUSEPPE                                1
BUONO SALVATORE                               1
DE RENZI ANNAMARIA                            1
LETIZIA FORTUNA                               1
PALAZZO RAFFAELE                              1
Name: Cliente Ticket Chiuso, Length: 278, dtype: int64
TROTTA DOMENICO            5
ESPOSITO GENNARO           5
RAN SRL                    5
GIOVISI SRL                4
MASSA GIOVANNI UTENZA 2    4
                          ..
IAVARONE LORENZO           1
GALLO GIUSEPPE             1
BUONO SALVATORE            1
DE RENZI ANNAMARIA         1
PALAZZO RAFFAELE           1
Name: Cliente Ticket Chiuso, Length: 278, dtype: int64
INFO:LOGGER_NAME:Get only Data from X_data_

In [24]:
print('Printing Polling Timestamp Min,Max :',X_data_cambium['Polling Timestamp'].values.min(),X_data_cambium['Polling Timestamp'].values.max())
print('Printing DataTime Min,Max :',X_data_ticket_closed['DataTime Ticket Chiuso'].values.min(),X_data_ticket_closed['DataTime Ticket Chiuso'].values.max())

Printing Polling Timestamp Min,Max : 2020-12-24 2021-03-22
Printing DataTime Min,Max : 2020-12-24 2021-03-24


# Dropping records where X_data_ticket_closed DataTime < minimum of Polling Timestamp

In [25]:
X_data_ticket_closed = X_data_ticket_closed[X_data_ticket_closed['DataTime Ticket Chiuso'] >= pd.to_datetime(X_data_cambium['Polling Timestamp'].values.min(),dayfirst = True, utc=True)]
print(X_data_ticket_closed.shape)
logger.info("Saving Dataset... Ticket")
X_data_ticket_closed.to_csv('./X_data_ticket_closed.csv', encoding='utf-8')

(403, 2)
INFO:LOGGER_NAME:Saving Dataset... Ticket


# Merging CPEs status and issues datasets

In [26]:
print("Dataset XY_data:",XY_data.shape)
print("Dataset X_data_ticket_closed:",X_data_ticket_closed.shape)
XY_data = pd.merge(XY_data,X_data_ticket_closed,left_on=['Device Name','Polling Timestamp'],right_on=['Cliente Ticket Chiuso','DataTime Ticket Chiuso'],how='left')
print ("Dataset XY_data:", XY_data.shape)
XY_data.to_csv('./XY_data.csv', encoding='utf-8')

Dataset XY_data: (32068, 24)
Dataset X_data_ticket_closed: (403, 2)
Dataset XY_data: (32068, 26)


# Adding "Ticket" label   

In [27]:
XY_data['Ticket'] = np.NaN
print(XY_data.shape)

print('Records with no Ticket:',XY_data['DataTime'].isnull().sum())
print('Records with Ticket:',XY_data['DataTime'].notnull().sum())

XY_data.loc[XY_data['DataTime'].isnull(),['Ticket']]  = '0'
XY_data.loc[XY_data['DataTime'].notnull(),['Ticket']] = '1'

print(XY_data.groupby('Ticket').size())

(32068, 27)
Records with no Ticket: 31889
Records with Ticket: 179
Ticket
0    31889
1      179
dtype: int64


In [29]:
# Impostare ticket = 1 per ogni cliente tutti i campioni che intercorrono tra data apertura e data chiusura ticket
'''

XY_data = XY_data.sort_values(by=['Device Name','Polling Timestamp','DataTime','DataTime Ticket Chiuso'])
print(XY_data)

XY_data_1 = XY_data.loc[(XY_data['Ticket'] == '1'),['Device Name','Polling Timestamp','DataTime']].sort_values(by=['Device Name','Polling Timestamp'])
print(XY_data_1)
XY_data_2 = XY_data.loc[(XY_data['DataTime Ticket Chiuso'].notnull()),['Device Name','Polling Timestamp','DataTime Ticket Chiuso']].sort_values(by=['Device Name','Polling Timestamp'])
print(XY_data_2)
XY_data.to_csv('./XY_data.csv', encoding='utf-8')
XY_data_1.to_csv('./XY_data_1.csv', encoding='utf-8')
XY_data_2.to_csv('./XY_data_2.csv', encoding='utf-8')


for index1,row1 in XY_data_1.iterrows():    
    for index, row in XY_data.iterrows():
        if((row['Device Name']==index1) & (row['DataTime Ticket Chiuso'].isnull()) & (row['DataTime'].isnull()) ) :
            XY_data.loc[index,['Y']]  = '2'
'''

"\n\nXY_data = XY_data.sort_values(by=['Device Name','Polling Timestamp','DataTime','DataTime Ticket Chiuso'])\nprint(XY_data)\n\nXY_data_1 = XY_data.loc[(XY_data['Ticket'] == '1'),['Device Name','Polling Timestamp','DataTime']].sort_values(by=['Device Name','Polling Timestamp'])\nprint(XY_data_1)\nXY_data_2 = XY_data.loc[(XY_data['DataTime Ticket Chiuso'].notnull()),['Device Name','Polling Timestamp','DataTime Ticket Chiuso']].sort_values(by=['Device Name','Polling Timestamp'])\nprint(XY_data_2)\nXY_data.to_csv('./XY_data.csv', encoding='utf-8')\nXY_data_1.to_csv('./XY_data_1.csv', encoding='utf-8')\nXY_data_2.to_csv('./XY_data_2.csv', encoding='utf-8')\n\n\nfor index1,row1 in XY_data_1.iterrows():    \n    for index, row in XY_data.iterrows():\n        if((row['Device Name']==index1) & (row['DataTime Ticket Chiuso'].isnull()) & (row['DataTime'].isnull()) ) :\n            XY_data.loc[index,['Y']]  = '2'\n"

# Load Customer file

In [30]:
data = './clienti/clienti_(1).csv'
colX = ['COD. CLIENTE','NOME CLIENTE','CLIENTE ATTIVO','COMUNE INSTALLAZIONE','DIFFUSIONE']

X_data_cliente = pd.read_csv(data, sep = ';', usecols= colX, encoding='utf-8') 
print(X_data_cliente.info())
print(X_data_cliente.shape)
print(X_data_cliente)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2067 entries, 0 to 2066
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   COD. CLIENTE          2067 non-null   object
 1   NOME CLIENTE          2067 non-null   object
 2   CLIENTE ATTIVO        2067 non-null   object
 3   COMUNE INSTALLAZIONE  2067 non-null   object
 4   DIFFUSIONE            1731 non-null   object
dtypes: object(5)
memory usage: 80.9+ KB
None
(2067, 5)
     COD. CLIENTE                       NOME CLIENTE CLIENTE ATTIVO  \
0            Fly1                    CSA s.c.a. r.l.             NO   
1            Fly2  STUDIO COMMERCIALE PALMIERO LUIGI             NO   
2            Fly3                   ALCAS ITALIA SRL             NO   
3            Fly4                   STAR ENERGIA SRL             SI   
4            Fly5                    BRICO APPIA SRL             NO   
...           ...                                ...          

# Delete records where 'NOME CLIENTE' occurrence > 2

In [31]:
print(X_data_cliente.info())
X_data_cliente = X_data_cliente.groupby('NOME CLIENTE')   # group by
X_data_cliente = (X_data_cliente.filter(lambda x: len(x) < 2))  #  HAVING COUNT(*) < 2
print(X_data_cliente.info())

#print(X_data_cliente.groupby('NOME CLIENTE').size().loc[lambda x: x<2].sort_values().)




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2067 entries, 0 to 2066
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   COD. CLIENTE          2067 non-null   object
 1   NOME CLIENTE          2067 non-null   object
 2   CLIENTE ATTIVO        2067 non-null   object
 3   COMUNE INSTALLAZIONE  2067 non-null   object
 4   DIFFUSIONE            1731 non-null   object
dtypes: object(5)
memory usage: 80.9+ KB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1946 entries, 0 to 2066
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   COD. CLIENTE          1946 non-null   object
 1   NOME CLIENTE          1946 non-null   object
 2   CLIENTE ATTIVO        1946 non-null   object
 3   COMUNE INSTALLAZIONE  1946 non-null   object
 4   DIFFUSIONE            1624 non-null   object
dtypes: object(5)
memory usage: 91.2+ KB
N

# More datasets merging (XY_data with X_data_cliente) 

In [32]:
print("Dataset XY_data:",XY_data.shape)
print("Dataset X_data_cliente:",X_data_cliente.shape)

#print(X_data_cliente ['NOME CLIENTE'].value_counts())
X_data_cliente ['NOME CLIENTE'] = X_data_cliente ['NOME CLIENTE'].str.strip()
X_data_cliente ['NOME CLIENTE'] = X_data_cliente ['NOME CLIENTE'].str.upper()
#print(X_data_cliente ['NOME CLIENTE'].value_counts())

XY_data_ctc_all = pd.merge(XY_data,X_data_cliente,left_on=XY_data['Device Name'].str.upper(),right_on=X_data_cliente['NOME CLIENTE'].str.upper(),how='left')

print ("Dataset XY_data_ctc_all:", XY_data_ctc_all.shape)

Dataset XY_data: (32068, 27)
Dataset X_data_cliente: (1946, 5)
Dataset XY_data_ctc_all: (32068, 33)


# Save Dataset 

In [33]:
logger.info("Saving Dataset...XY_data_ctc_all")
XY_data_ctc_all.to_csv('./XY_data_ctc_all.csv', encoding='utf-8')

INFO:LOGGER_NAME:Saving Dataset...XY_data_ctc_all


# Load Dataset from previous saving 

Features are: Polling Timestamp,MAC,Device Name, Device Type, Device Mode, Downlink MCS, Uplink MCS, Downlink ThroughPut (Kbps), Uplink ThroughPut (Kbps), Downlink Retransmission Pkts (%),
Uplink Retransmission Pkts (%), CPUs, Session Drops, Downlink SNR (dB), Uplink SNR (dB),
Downlink Usage (Packets), Uplink Usage (Packets), Downlink RSSI (dBm), Uplink RSSI (dBm),
Offline Count, Status, Status Duration, COD. CLIENTE, COMUNE INSTALLAZIONE, DIFFUSIONE

In [35]:
data = './XY_data_ctc_all.csv'
colX = ['Polling Timestamp', 'MAC', 'Device Name', 'Device Type', 'Device Mode',
        'Downlink MCS', 'Uplink MCS', 'Downlink ThroughPut (Kbps)', 'Uplink ThroughPut (Kbps)',
        'Downlink Retransmission Pkts (%)', 'Uplink Retransmission Pkts (%)', 'CPUs',
        'Session Drops', 'Downlink SNR (dB)', 'Uplink SNR (dB)', 'Downlink Usage (Packets)',
        'Uplink Usage (Packets)', 'Downlink RSSI (dBm)', 'Uplink RSSI (dBm)', 'Offline Count',
        'Status', 'MStatus Duration','COD. CLIENTE','COMUNE INSTALLAZIONE','DIFFUSIONE', 'CLIENTE ATTIVO', 'Ticket']

XY_data_ctc = pd.read_csv(data, sep = ',', usecols= colX, encoding='utf-8') 
print(XY_data_ctc.info())
print(XY_data_ctc.shape)
print(XY_data_ctc)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32068 entries, 0 to 32067
Data columns (total 27 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Polling Timestamp                 32068 non-null  object 
 1   MAC                               32068 non-null  object 
 2   Device Name                       32068 non-null  object 
 3   Device Type                       32068 non-null  object 
 4   Device Mode                       32068 non-null  object 
 5   Downlink MCS                      32068 non-null  object 
 6   Uplink MCS                        32068 non-null  object 
 7   Downlink ThroughPut (Kbps)        32068 non-null  float64
 8   Uplink ThroughPut (Kbps)          32068 non-null  float64
 9   Downlink Retransmission Pkts (%)  32068 non-null  float64
 10  Uplink Retransmission Pkts (%)    32068 non-null  float64
 11  CPUs                              32068 non-null  float64
 12  Sess

# Saving Dataset 

In [36]:
logger.info("Saving Dataset...XY_data_ctc")
XY_data_ctc.to_csv('./XY_data_ctc.csv', encoding='utf-8')

INFO:LOGGER_NAME:Saving Dataset...XY_data_ctc


# Pre-elaborazione
## rimozione e imputazione dei valori mancanti
## gestione dati ordinali
## gestione dati nominali
## codifica delle etichette delle classi
## dataset di addestramento e dataset di test
## riduzione in scala e normalizzazione
## selezione delle caratteristiche appropriate 

# Gestione Valori Mancanti (GVM)

In [37]:
# Print the number of null (missing) values for each column
print(XY_data_ctc.isnull().sum())
# Print the occurrence of null values (true if data is missing)
print(XY_data_ctc.isnull())

Polling Timestamp                      0
MAC                                    0
Device Name                            0
Device Type                            0
Device Mode                            0
Downlink MCS                           0
Uplink MCS                             0
Downlink ThroughPut (Kbps)             0
Uplink ThroughPut (Kbps)               0
Downlink Retransmission Pkts (%)       0
Uplink Retransmission Pkts (%)         0
CPUs                                   0
Session Drops                          0
Downlink SNR (dB)                      0
Uplink SNR (dB)                        0
Downlink Usage (Packets)               0
Uplink Usage (Packets)                 0
Downlink RSSI (dBm)                    0
Uplink RSSI (dBm)                      0
Offline Count                          0
Status                                 0
MStatus Duration                       0
Ticket                                 0
COD. CLIENTE                        7726
CLIENTE ATTIVO  

In [38]:
# I check how many samples of the feature COD. CLIENTE are null
print('No. of samples with COD. CLIENTE not completed:',XY_data_ctc['COD. CLIENTE'].isnull().sum())
print((XY_data_ctc[XY_data_ctc['COD. CLIENTE'].isnull()]).groupby(['Device Name']).size())
# Drop only the lines where COD. CLIENTE is null
XY_data_ctc.dropna(subset=['COD. CLIENTE'],inplace=True)
print ('Dataset after deletion of samples with COD. CLIENTE not completed:\n', XY_data_ctc.shape)
print('No. of samples with COD. CLIENTE not completed:',XY_data_ctc['COD. CLIENTE'].isnull().sum())

No. of samples with COD. CLIENTE not completed: 7726
Device Name
ABBATE ANGELO                    68
ABBATE ANGELO LO. STAZZA         68
ABBATE PASQUALE - SOLO VOIP      68
AGAPE GHEORGHITAN STEFAN         68
AIRONE SERVICES SRL              68
                                 ..
TARTAGLIONE VINCENZO DEPOSITO    69
TRYBULSKI                        68
UBALDO NERO                      68
UCCIERO ANTONIO                  40
UCR SOC COOP                     68
Length: 129, dtype: int64
Dataset after deletion of samples with COD. CLIENTE not completed:
 (24342, 27)
No. of samples with COD. CLIENTE not completed: 0


# Drop records where customer is not active

In [39]:
print('No. of samples with CLIENTE ATTIVO not completed:',XY_data_ctc['CLIENTE ATTIVO'].isnull().sum())
print(XY_data_ctc.groupby('CLIENTE ATTIVO').size())

XY_data_ctc = XY_data_ctc[XY_data_ctc['CLIENTE ATTIVO'] == 'SI']
print(XY_data_ctc.groupby('CLIENTE ATTIVO').size())
print(XY_data_ctc.info())
print(XY_data_ctc.shape)
print('No. of samples with CLIENTE ATTIVO not completed:',XY_data_ctc['CLIENTE ATTIVO'].isnull().sum())

No. of samples with CLIENTE ATTIVO not completed: 0
CLIENTE ATTIVO
NO      748
SI    23594
dtype: int64
CLIENTE ATTIVO
SI    23594
dtype: int64
<class 'pandas.core.frame.DataFrame'>
Int64Index: 23594 entries, 0 to 32067
Data columns (total 27 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Polling Timestamp                 23594 non-null  object 
 1   MAC                               23594 non-null  object 
 2   Device Name                       23594 non-null  object 
 3   Device Type                       23594 non-null  object 
 4   Device Mode                       23594 non-null  object 
 5   Downlink MCS                      23594 non-null  object 
 6   Uplink MCS                        23594 non-null  object 
 7   Downlink ThroughPut (Kbps)        23594 non-null  float64
 8   Uplink ThroughPut (Kbps)          23594 non-null  float64
 9   Downlink Retransmission Pkts (%)  23594 non-null 

In [40]:
# Dropping Polling Timestamp, Device Mode, CLIENTE ATTIVO

print('Dataset before dropping columns :',XY_data_ctc.shape)
XY_data_ctc.drop(columns=['Polling Timestamp', 'Device Mode', 'CLIENTE ATTIVO'],inplace=True)
print ('Dataset after dropping columns :', XY_data_ctc.shape)
print(XY_data_ctc.info())

Dataset before dropping columns : (23594, 27)
Dataset after dropping columns : (23594, 24)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 23594 entries, 0 to 32067
Data columns (total 24 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   MAC                               23594 non-null  object 
 1   Device Name                       23594 non-null  object 
 2   Device Type                       23594 non-null  object 
 3   Downlink MCS                      23594 non-null  object 
 4   Uplink MCS                        23594 non-null  object 
 5   Downlink ThroughPut (Kbps)        23594 non-null  float64
 6   Uplink ThroughPut (Kbps)          23594 non-null  float64
 7   Downlink Retransmission Pkts (%)  23594 non-null  float64
 8   Uplink Retransmission Pkts (%)    23594 non-null  float64
 9   CPUs                              23594 non-null  float64
 10  Session Drops                     23594

# Gestione di dati Categorici : Mappaggio di caratteristiche Ordinali
    Downlink MCS, Uplink MCS

In [41]:
XY_data_ctc['Downlink MCS'] = XY_data_ctc['Downlink MCS'].str.strip()
XY_data_ctc['Downlink MCS'] = XY_data_ctc['Downlink MCS'].str.upper()
print ('Unique values Downlink MCS:\n', XY_data_ctc['Downlink MCS'].value_counts())
XY_data_ctc['Uplink MCS'] = XY_data_ctc['Uplink MCS'].str.strip()
XY_data_ctc['Uplink MCS'] = XY_data_ctc['Uplink MCS'].str.upper()
print ('Unique values Uplink MCS\n', XY_data_ctc['Uplink MCS'].value_counts())
print(XY_data_ctc.shape)

size_mapping = {
        'DS 9': 35,
        'DS 8': 34,
        'DS 7': 33,
        'DS 6': 32,
        'DS 5': 31,
        'DS 4': 30,
        'DS 3': 29,
        'DS 2': 28,
        'DS 1': 27,
        'DS 0': 26,
        'SS 9': 25,
        'SS 8': 24,
        'SS 7': 23,
        'SS 6': 22,
        'SS 5': 21,
        'SS 4': 20,
        'SS 3': 19,
        'SS 2': 18,
        'SS 1': 17,
        'SS 0': 16,
        '15': 15,
        '14': 14,
        '13': 13,
        '12': 12,
        '11': 11,
        '10': 10,
        '9': 9,
        '8': 8,
        '7': 7,
        '6': 6,
        '5': 5,
        '4': 4,
        '3': 3,
        '2': 2,
        '1': 1,
        '0': 0}
XY_data_ctc['Downlink MCS'] = XY_data_ctc['Downlink MCS'].map(size_mapping)
print ('Unique values Downlink MCS:\n', XY_data_ctc['Downlink MCS'].value_counts())
XY_data_ctc['Uplink MCS'] = XY_data_ctc['Uplink MCS'].map(size_mapping)
print ('Unique values Uplink MCS\n', XY_data_ctc['Uplink MCS'].value_counts())

Unique values Downlink MCS:
 14      4126
12      3113
DS 8    3077
13      2846
11      2036
15      1627
10       964
6        825
DS 7     686
DS 9     552
4        534
5        502
9        483
3        359
0        345
DS 6     329
8        313
DS 5     178
2        159
DS 4     147
7        138
DS 3      68
DS 0      42
SS 8      32
SS 9      31
DS 2      23
1         22
DS 1      22
SS 7       8
SS 6       3
SS 5       2
SS 4       2
Name: Downlink MCS, dtype: int64
Unique values Uplink MCS
 8       5837
1       4562
9       1476
2       1255
DS 0    1005
7        759
12       726
11       685
3        620
DS 1     522
DS 2     498
DS 7     475
10       455
SS 9     448
DS 3     425
6        411
14       394
DS 6     371
0        367
4        365
DS 5     325
SS 8     290
13       259
5        217
DS 4     197
DS 8     130
SS 7     124
SS 6     112
SS 1     110
SS 4      73
SS 3      58
SS 5      22
SS 2      17
15         4
Name: Uplink MCS, dtype: int64
(23594, 24)
Unique valu

# Gestione di dati Categorici : Nominale
 Codifica delle Caratteristiche nominali (One-Hot) in valori interi. 
 MAC, Device Name, Device Type, Status, COD. CLIENTE, COMUNE INSTALLAZIONE, DIFFUSIONE


In [42]:

print(XY_data_ctc.shape)
print ('Unique values MAC:\n', XY_data_ctc ['MAC'].value_counts())
XY_data_ctc = pd.get_dummies(XY_data_ctc, columns = ['MAC'])
print(XY_data_ctc.shape)

print ('Unique values Device Name:\n', XY_data_ctc ['Device Name'].value_counts())
XY_data_ctc = pd.get_dummies(XY_data_ctc, columns = ['Device Name'])
print(XY_data_ctc.shape)

print ('Unique values Device Type:\n', XY_data_ctc['Device Type'].value_counts())
XY_data_ctc = pd.get_dummies(XY_data_ctc, columns = ['Device Type'])
print(XY_data_ctc.shape)

print ('Unique values Status:\n', XY_data_ctc['Status'].value_counts())
XY_data_ctc = pd.get_dummies(XY_data_ctc, columns = ['Status'])
print(XY_data_ctc.shape)

print ('Valori univoci COD. CLIENTE:\n', XY_data_ctc['COD. CLIENTE'].value_counts())
XY_data_ctc = pd.get_dummies(XY_data_ctc, columns = ['COD. CLIENTE'])
print(XY_data_ctc.shape)

print ('Unique values COMUNE INSTALLAZIONE:\n', XY_data_ctc['COMUNE INSTALLAZIONE'].value_counts()) 
XY_data_ctc = pd.get_dummies(XY_data_ctc, columns = ['COMUNE INSTALLAZIONE'])
print(XY_data_ctc.shape)

print ('Unique values DIFFUSIONE:\n', XY_data_ctc['DIFFUSIONE'].value_counts()) 
XY_data_ctc = pd.get_dummies(XY_data_ctc, columns = ['DIFFUSIONE'])
print(XY_data_ctc.shape)

print(XY_data_ctc.info())


(23594, 24)
Unique values MAC:
 80:2A:A8:E5:A9:12    71
80:2A:A8:B3:39:A2    71
80:2A:A8:E3:74:80    71
E4:8D:8C:ED:76:38    71
DC:9F:DB:4D:39:9A    71
                     ..
00:04:56:23:78:B8     7
58:C1:7A:45:E5:AC     7
58:C1:7A:45:E8:1E     6
58:C1:7A:4D:1E:32     5
58:C1:7A:4D:22:9C     4
Name: MAC, Length: 381, dtype: int64
(23594, 404)
Unique values Device Name:
 NOVIELLO LEONARDO          136
DIANA RAFFAELLA             86
TAVOLETTA SILVIA            76
RUSSO ROSARIO               76
ESPOSITO NICOLE             76
                          ... 
ZAGARIA RAFFAELE             9
DE CRESCENZO DOMENICO        7
MARTINELLI GERARDO           7
CILIENTO MARIA GIOVANNA      6
CORVINO GIANLUCA             4
Name: Device Name, Length: 366, dtype: int64
(23594, 769)
Unique values Device Type:
 ePMP Force 180       11213
ePMP Force 300-16     5252
ePMP Elevate          4056
ePMP Force 200        2791
ePMP Force 190         204
ePMP Force 300-25       78
Name: Device Type, dtype: int64
(2359

# Apprendimento
## Selezione modello predittivo
## scelta metrica prestazionale
## regolarizzazione

# Valutazione modello finale

# Predizione